In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#data
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Data/quora duplicate identification/train.csv")

In [ ]:
import numpy as np

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
len(np.intersect1d(list(set(('What is the step by step guide to invest in sh...').split())),list(set(('What is the step by step guide to invest in sh...').split()))))

10

In [ ]:
list(set(('What is the step by step guide to invest in sh...').split()))

['by', 'the', 'sh...', 'step', 'to', 'is', 'What', 'guide', 'invest', 'in']

In [ ]:
x1_data=df['question1'].astype(str).values
x2_data=df['question2'].astype(str).values

In [ ]:
x1_data,x2_data=[i.lower() for i in x1_data],[i.lower() for i in x2_data]

In [ ]:
from drive.MyDrive.regex import regex_clean

x1_data=[regex_clean(data) for data in x1_data]
x2_data=[regex_clean(data) for data in x2_data]

In [ ]:
(x1_data[0]+" "+x2_data[0]).split()

['what',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india',
 'what',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market']

### Only one question pair has been tested,similarly any question pair can can be tested and score can be produced,the question pair with best fuzzy score is considered to be carried forward in the Machine learning system's pipeline  

In [ ]:
q11,q22=[],[]
overlap=[]
for q1,q2 in zip(x1_data,x2_data):
  q1="the verbal aptitude questions have gramatical mistakes in the question and we are asked to choose the options with gramatical mistake" #
  q2="few words missing in them, I hope these can be looked into gramatical mistakes in the question so that the next test is not so"
  c=np.intersect1d(list(set(q1.split())),list(set(q2.split())))
  if len(c)==0:
    fuzzy_ratio=0
  else:
    overlap.append(len(c))
    for word in (q1+" "+q2).split():
      if word not in set(q2.split()):
        q11.append(word)

      if word not in set(q1.split()):
        q22.append(word)

    rem_q11,rem_q22=[],[]
    p_overlap=0
    break

        

In [ ]:
q11,q22  # for 1st question pair

(['verbal',
  'aptitude',
  'questions',
  'have',
  'and',
  'we',
  'are',
  'asked',
  'to',
  'choose',
  'options',
  'with',
  'mistake'],
 ['few',
  'words',
  'missing',
  'them,',
  'I',
  'hope',
  'these',
  'can',
  'be',
  'looked',
  'into',
  'so',
  'that',
  'next',
  'test',
  'is',
  'not',
  'so'])

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=148896 sha256=5009f2cb5ac34702bf66c3d2417adfe7c089bc3a5dee5d1789180be4abaee208
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# lets use google's word2vec embeddings as mentioned in the paper
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download alvations/vegetables-google-word2vec

 99% 1.64G/1.66G [00:17<00:00, 145MB/s]
100% 1.66G/1.66G [00:17<00:00, 99.3MB/s]


In [ ]:
!unzip /content/vegetables-google-word2vec.zip

Archive:  /content/vegetables-google-word2vec.zip
  inflating: word2vec.news.negative-sample.300d.npy  
  inflating: word2vec.news.negative-sample.300d.pkl  
  inflating: word2vec.news.negative-sample.300d.txt  


In [ ]:
embedding=np.load('/content/word2vec.news.negative-sample.300d.npy')

In [ ]:
vocab=[line.strip() for line in open('/content/word2vec.news.negative-sample.300d.txt')]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([embedding[vocab.index('how')]],[embedding[vocab.index('me')]]) #checking 

array([[0.28679878]], dtype=float32)

In [ ]:
#paper has used normalized levenshtein distance and the optimum value of thres1 and thres2 were calculated after several experiments by researchers

thres1,thres2=0.6,0.55

from Levenshtein import distance                             

for word1 in q11:
  for word2 in q22:
    dist= distance(word1,word2)
    norm_dist=np.linalg.norm(dist)
    sync_score=(1-norm_dist)
    if sync_score>thres1:
      print(f'syntactic similarity {word1,word2}/n')
      p_overlap+=sync_score
      rem_q11.append(word1)
      rem_q22.append(word2)
      break
    elif word1 in vocab and word2 in vocab:                                 
      a=embedding[vocab.index(word1)]                                       # averaging the embedding vectors will loose information about the word and just summing it up is not a soln
      b=embedding[vocab.index(word2)]                                       # I have used sklearn metrics for measuring cosine similarity between two embeeding vecors of given two words
      sym_score=cosine_similarity([a],[b])
      if sym_score>thres2:
        p_overlap+=sym_score
        print(f'symantic similarity {word1,word2}')
        rem_q11.append(word1)
        rem_q22.append(word2)       
        break
    else:
      continue




symantic similarity ('we', 'I')
symantic similarity ('are', 'these')


In [ ]:
rem_q11,rem_q22

(['we', 'are'], ['I', 'these'])

Just playing with word vectors


In [ ]:
test=sum(embedding[vocab.index('me')])
test

-1.1650524139404297

In [ ]:
np.dot(test,test)/(2*np.linalg.norm(test))

0.5825262069702148

In [ ]:
assert np.dot(test,test)/2*np.linalg.norm(test) > 0.55     # check if it surpassses the thres2 for semantic similarity(cosine similarity) 

In [ ]:
l1=len([word for word in q11 if word not in rem_q11])
l2=len([word for word in q11 if word not in rem_q22])

In [ ]:
l1,l2

(11, 13)

In [ ]:
fuzzy_ratio=(len(c)+p_overlap)/(l1+l2+len(c)+len(rem_q11))

In [ ]:
fuzzy_ratio

array([[0.20267603]], dtype=float32)

In [ ]:
p_overlap

array([[1.2829573]], dtype=float32)